as always: **do not edit in tsds folder**

In [ ]:
# Load data
import pandas as pd 
import numpy as np
import gensim
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.corpus import stopwords, reuters
# List of document ids
documents = reuters.fileids()
 
# Load training data and test data
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))
# Load categories

import pandas as pd
import numpy as np
categories = reuters.categories()
cat2index = {cat:num for num, cat in enumerate(categories)}
def process_categories(cats):
    "This function takes a list of named categories and converts them into a sparse indicator vector."
    vec = np.zeros(len(cat2index))
    for cat in cats:
        idx = cat2index[cat]
        vec[idx] = 1
    return vec
def make_y(fileids):
    y_mat = []
    for doc in fileids:
        y_mat.append(process_categories(reuters.categories(doc)))
    return pd.DataFrame(y_mat,columns=categories)
y_df_train = make_y(train_docs_id)
y_df_test = make_y(test_docs_id)
# Documents
train_docs = [reuters.raw(doc_id) for doc_id in train_docs_id]
test_docs = [reuters.raw(doc_id) for doc_id in test_docs_id]
# lemmatizing
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize(words):
    pos_doc = nltk.pos_tag(words)
    pos_doc = [(word,get_wordnet_pos(tag)) for word,tag in pos_doc]
    return [lemmatizer.lemmatize(word,tag) if tag!='' else word for word,tag in pos_doc]
#### NORMALIZATION+TOkenization
# Define stopwords
from nltk.corpus import stopwords
stop_words_nltk = set(stopwords.words('english')) # remember to define as set for performance
#own_stop_words = set(["the","it","she","he", "a","trump",'idiot','failure','weak']) # Define your own list of stopwords

#The stemmers and lemmers need to be initialized before being run
porter = nltk.stem.porter.PorterStemmer() # 
snowball = nltk.stem.snowball.SnowballStemmer('english') # a greedy stripper
# Normalization using a Class 
class custom_tokenizer():
    def __init__(self,lowercase=False
                     ,remove_non_alpha=False
                     , stopwords = False
                     , stemmer = False
                     , lemmer = False,
                     tokenizer = nltk.word_tokenize):
        self.lowercase = lowercase
        self.tokenizer = tokenizer
        self.remove_non_alpha = remove_non_alpha
        self.lemmer = lemmer
        self.stemmer = stemmer
        self.stopwords = stopwords
        
    def tokenize(self,string):
        ####### Optional ######
        # Filter unwanted elements, or substitute elements using regex here.
        ## Examples: 
        ### different URLS substituted for one __link__ token
        ### numbers substituted __number__ token, or separate tokens __zipcode__, __phonenumber__, and __money__.

        # preprocess clean strings.
        ####### Optional ######

        # Split strings into words.
        words = self.tokenizer(string)
        # Lemmatizing
        if self.lemmer:
            words = lemmatize(words)
        #removing non-words
        if self.remove_non_alpha:
            words = [w for w in words if w.isalpha()]

        # lowering
        if self.lowercase:
            words = [w.lower() for w in words]
        #Now we can use the semmer, if provided
        if self.stemmer:
            words = [self.stemmer.stem(w) for w in words]

        #And remove the stopwords
        if self.stopwords:
            words = [w for w in words if w not in self.stopwords]
        #We will return a list with the stopwords removed
        return words


tokenizer = custom_tokenizer(**default) # change configurations here
gram = 2 # define number of ngrams

vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1,gram+1),min_df=3,tokenizer=tokenizer.tokenize)
# load documents
vectorizer.fit(train_docs)
mat_train = vectorizer.transform(train_docs)
mat_test = vectorizer.transform(test_docs)


 # Implementation of the Naive Bayes


**ex.1.1**
Make your own implementation of the Naive Bayes classifier following the steps defined by Christopher Manning in his IR book:
https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html
**ex.1.2**
Compare your classifier with the sklearn implementation on the reuters dataset on the following parameters:
* Accuracy
* Training time using the `%timeit` command.

***ex.1.3***
Install the code profilling tool: line_profiler
     `conda install line_profiler`
     
And use this tool to profile your own classifier:

`%load_ext line_profiler
%lprun -f NB.fit NB.fit(x_train,y_train)`

***extra***
Optimize your implementation. ***Hint**: Can you create more matrix operations?


In [1]:
def NB(object): 
    def __init__(self,smoothing=1):
        # laplace smoothing
        self.smoothing = smoothing
        
    def fit(self,x_train,y_train):
        """This function will use the training data to estimate p(x_i|C_i) for all features and all categories.
        As described by: Christopher Manning in https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html"""
    def predict(x_test):
        "This function will do the inference and return the argmax of P(x|C_i). Also described in Mannings book."

# Analyze test results
We are gonna use our simulation results to learn about each classifier.
**ex.2.1**:
    - Plot the learning curve of each classifier.
        - Broken down on each task.
        - Are the classifiers saturated or do they need more samples?
        - Does it vary with the vocab_size?

**ex.2.2**
    - Investigate inference time for each classifier in relation to training_size and vocab_size.

**ex.2.3**:
    - Investigate training_time for each classifier in relation to the vocab_size.

    
    

In [113]:
import pandas as pd
test_df = pd.read_csv('test_results2.csv')
test_df.sample(10)


,accuracy,category,clf,f1,infer_time,ngram,precision,preprocessing,recall,train_time,training_size,vocab_size
836,0.934415,money-fx,nb,0.331081,0.038709,2,0.418803,"{""lowercase"": true, ""stopwords"": true, ""lemmer...",0.273743,0.048521,3500,50995
1050,0.953958,acq,svm,0.898909,0.002929,2,0.942073,"{""lowercase"": true, ""stopwords"": true, ""remove...",0.859527,0.007315,500,38388
624,0.984432,earn,svm,0.978470,0.002865,1,0.974453,"{""lowercase"": true, ""stopwords"": true, ""lemmer...",0.982521,0.178637,4500,11090
1021,0.932428,earn,knn,0.910839,1.407339,1,0.867610,"{""lowercase"": true, ""stopwords"": true, ""remove...",0.958602,0.004258,5000,9751
186,0.970520,money-fx,svm,0.729483,0.003352,2,0.800000,"{""lowercase"": true, ""stopwords"": true}",0.670391,0.259074,2500,51365
266,0.794634,earn,knn,0.770880,1.062056,2,0.644225,"{""lowercase"": true, ""stopwords"": true}",0.959522,0.004228,2500,51365
473,0.988076,earn,svm,0.983395,0.004324,2,0.986124,"{""lowercase"": true, ""stopwords"": true, ""stemme...",0.980681,0.499916,4000,51169
623,0.983107,earn,svm,0.976616,0.002128,1,0.973492,"{""lowercase"": true, ""stopwords"": true, ""lemmer...",0.979761,0.350015,4000,11090
19,0.982445,earn,svm,0.975655,0.001375,1,0.974312,"{""lowercase"": true, ""stopwords"": true}",0.977001,0.028363,2000,13131
237,0.934084,money-fx,nb,0.364217,0.015915,2,0.425373,"{""lowercase"": true, ""stopwords"": true}",0.318436,0.028289,4000,51365


In [114]:
import json
# First we will rename the preprocessing column
def rename_preprocessing(js):
    "This function will remove the default settings, that was not experimented with"
    js = json.loads(js)
    del js['stopwords']
    del js['lowercase']
    return json.dumps(js).strip('{} ')
test_df['preprocessing'] = test_df.preprocessing.apply(rename_preprocessing)

# Vectorization of large datasets
When working with large datasets vectorization cannot be done with all data loaded into memory at once. An optimal indexing (i.e. the most prevalent words are assigned to smaller indexes) is therefore inefficient since it would require means running over the data twice. Instead one can transform each input and index on the fly.

**ex.3.1** In this exercise I want you to implement a vectorization scheme as described here: https://cmry.github.io/notes/ngrams

**ex.3.2** Apply this function to the Reuters data, but instead of loading it all at the same time I want you to design an interator that reads each sample from disk and applies your new vectorization scheme.

***extra*** 
For each 1000 samples, I want you to do online training of a classifier using the partial_fit function of a sklearn classifier: e.g. http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB.partial_fit.
